In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,...,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand
153,AFG,Asia,Afghanistan,2020-06-11,22143,684,405,21,568.814,17.571,...,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.50
248,ALB,Europe,Albania,2020-06-11,1341,42,34,0,465.981,14.594,...,13.188,8.643,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89
407,DZA,Africa,Algeria,2020-06-11,10484,102,732,8,239.082,2.326,...,6.211,3.857,13913.839,0.5,278.364,6.73,0.7,30.4,83.741,1.90


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-06-10,France,154858.0,0.0,29360.0,0.0,0.0,0.0
0,2020-06-10,China,84254.0,0.0,4651.0,0.0,0.0,0.0
0,2020-06-10,Italy,235972.0,0.0,34128.0,0.0,0.0,0.0
0,2020-06-10,Spain,242199.0,0.0,27148.0,0.0,0.0,0.0
0,2020-06-10,United States,2001597.0,0.0,112689.0,0.0,0.0,0.0
0,2020-06-10,World,7349112.0,0.0,415103.0,0.0,0.0,0.0
0,2020-06-10,United Kingdom,290751.0,0.0,41009.0,0.0,0.0,0.0
0,2020-06-10,Germany,185288.0,0.0,8773.0,0.0,0.0,0.0
0,2020-06-10,Iran,178288.0,0.0,8499.0,0.0,0.0,0.0
0,2020-06-10,Turkey,173060.0,0.0,4747.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-06-11,France,154858.0,155136.0,29360.0,29319.0,-278.0,41.0
0,2020-06-11,China,84254.0,84209.0,4651.0,4638.0,45.0,13.0
0,2020-06-11,Italy,235972.0,235763.0,34128.0,34114.0,209.0,14.0
0,2020-06-10,Spain,242199.0,242280.0,27148.0,27136.0,-81.0,12.0
0,2020-06-11,United States,2001597.0,2000464.0,112689.0,112924.0,1133.0,-235.0
0,2020-06-11,World,7349112.0,7343562.0,415103.0,416430.0,5550.0,-1327.0
0,2020-06-11,United Kingdom,290751.0,290143.0,41009.0,41128.0,608.0,-119.0
0,2020-06-11,Germany,185288.0,185416.0,8773.0,8755.0,-128.0,18.0
0,2020-06-11,Iran,178288.0,177938.0,8499.0,8506.0,350.0,-7.0
0,2020-06-11,Turkey,173060.0,173036.0,4747.0,4746.0,24.0,1.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")